# Analyser des données depuis des API Web

Python permet de s'interfacer avec de nombreux services web (API). 

Dans cette partie nous verrons : 

- Ce qu'est une API 
- Comment Python et Pandas permettent de transformer des données brutes en un DataFrame exploitable

## Introduction aux API Web

Une API (Application Programming Interface) permet de récupérer des des données mises à disposition par des services Web accessibles depuis une adresse particulière. 

La plupart des services Web (Google Analytics, Mixpanel, Twitter, Facebook, le New York Times etc..) dispose d'API ce qui en fait une source de données de choix. 

Python et Pandas sont d'excellents outils pour récupérer, transformer et analyser ces données web. 

### URLs 

Une URL (Unique Ressource Locator) permet d'identifier de manière unique une ressource sur Internet. Prenons l'URL suivante : 

Cette URL se décompose en blocs logiques : 

- ***http://*** = le schéma, détermine le protocole que nous utilisons pour communiquer avec le service Web. Nous utiliserons principalement le protocole **http**
- ***www.example.com*** = l'hôte qui détermine sur quel serveur de l'internet nous allons communiquer
- ***/foo/bar*** = spécifie le chemin où se trouve la ressource
- ***?arg1=baz&arg2=quux*** les paramètres de la requête

### HTML vs JSON

- HTML permet de renvoyer de la donnée pour un utilisateur. Lorsqu'une requête est faite sur un serveur, celui-ci renvoie des éléments (html, css,js, etc..) permettant au navigateur d'afficher une page web **user-friendly**

<img src='files/images/html.png' width=500>

- JSON permet de renvoyer de la donnée structurée, exploitable par une machine. Lorsqu'un service doit communiquer avec un autre service, les données renvoyées sont **machine-friendly**. Plusieurs formats sont possibles (CSV, JSON, XML etc..)

<img src='files/images/json.png' width=500>

Le format JSON (Javascript Object Notation) est un des formats préférés des Web API. 

Il peut être composé de deux structures : 

- Dictionnaire clé / valeur
- Tableaux d'objets 

Voici un exemple de JSON : 

In [ ]:
{
     "firstName": "John",
     "lastName": "Smith",
     "address": {
         "streetAddress": "21 2nd Street",
         "city": "New York",
         "state": "NY",
         "postalCode": 10021
     },
     "phoneNumbers": [
         "212 555-1234",
         "646 555-4567"
     ]
 }

### Exemples d'API populaires

La plupart des services Web disposent d'API. Elles sont également documentées voici quelques exemples : 

- Facebook - https://developers.facebook.com/tools/explorer/
- Twitter - https://dev.twitter.com/rest/tools/console
- Google Analytics - https://ga-dev-tools.appspot.com/explorer/
- The New York Times - http://developer.nytimes.com/docs

## Let's do it ! Analyse des tweets du compte officiel de l'Elysée

Avec près de 500 millions de tweets émis chaque jour, Twitter est une source de données inépuisable. 

Les données sont accessibles au format JSON par une API bien documentée. 

L'analyse des tweets offre plusieurs possibilités : 

- Analyse temporelle (date et heure d'émission)
- Analyse texte, de sentiment
- Analyse du nombre de retweets
- Analyse de graphe (réponse, mentions etc..)

Dans cet exercice, nous allons faire une analyse descriptive du compte Twitter officiel de l'Elysée (https://twitter.com/Elysee). 

Nous allons essayer de répondre à plusieurs questions : 

- Comment évolue le nombre de tweets sur le compte officiel ? 
- Y a t il des pics de retweets ? Si oui pourquoi ? 
- Quel sont les comptes que l'Elysée mentionne le plus ? 
- Quels sont les thèmes / hashtags les plus populaires ? 
- Comment ces thèmes évoluent ils dans le temps ? 

### Se connecter à l'API de Twitter

De nombreuses librairies Python ont été développées pour "dialoguer" avec l'API de Twitter. 

Une liste officielle des librairies Twitter par langage peut être trouvée à cette adresse - https://dev.twitter.com/overview/api/twitter-libraries
    
Pour les besoins de cet exercice, nous utiliserons la librairie **TwitterSearch** - https://github.com/ckoepp/TwitterSearch. Une documentation exhaustive des méthodes de cette librairie est disponible à l'adresse - https://twittersearch.readthedocs.org/en/latest/TwitterSearch.html

Installez TwitterSearch depuis votre terminal en exécutant : 

In [ ]:
pip install TwitterSearch

L'accès à l'API est sécurisé. Pour obtenir des clés d'accès : 

- Créez un compte Twitter
- Créez une application sur https://apps.twitter.com/

On peut dès lors appeler la librarie TwitterSearch en spécifiant ses identifiants de connexion : 

In [ ]:
from TwitterSearch import *

twitter_api = TwitterSearch(
    consumer_key = '¨* ',
    consumer_secret = '*',
    access_token = '*',
    access_token_secret = '*'
    )

Nous allons faire un premier test, en recherchant tous les tweets qui contiennent un mot en particulier. Nous utilisons pour cela la méthode **search_tweets** de l'objet **TwitterSearchOrder()**

In [ ]:
tso = TwitterSearchOrder() # nous créons ici un objet TwitterSearchOrder()
tso.set_keywords(['Elysee']) # recherchons tous les tweets comportant le mot Elysee
        
search_results = twitter_api.search_tweets(tso) # La méthode search_tweets permet de rechercher tous les tweets 

La réponse de l'API est au format JSON. Nous pouvons explorer le contenu, en utilisant la méthode **keys()**

In [ ]:
print(search_results.keys())

print(search_results['content'].keys())

In [ ]:
print(search_results)

In [ ]:
print(len(search_results['content']['statuses']))

L'objet'statuses' est une liste de 100 éléments.

Nous pouvons également afficher les clés de chacun des tweets renvoyés : 

In [ ]:
print(search_results['content']['statuses'][99]['text'])

### Charger et inspecter les données

In [ ]:
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
from datetime import datetime

In [ ]:
%matplotlib inline
plt.rc('figure', figsize=(20, 5))

Si nous souhaitons analyser l'intégralité des tweets d'un compte en particulier, nous devons utiliser l'objet **TwitterUserOrder** :

In [ ]:
tuo = TwitterUserOrder('Elysee')

elysee_results = twitter_api.search_tweets(tuo)

Nous pouvons afficher le premier tweet renvoyé par notre requête via : 

In [ ]:
elysee_results['content'][0]

Il nous faut donc réaliser une **boucle**. Le temps pour récupérer les données pouvant être long, on pourra utiliser le fichier **elysee.txt** qui contient un export des tweets bruts. 

In [ ]:
from TwitterSearch import *

data = []

try:
    tuo = TwitterUserOrder('Elysee') 

    twitter_api = TwitterSearch(
        consumer_key = 'XXX',
        consumer_secret = 'XXX',
        access_token = 'XXX',
        access_token_secret = 'XXX'
    )
    
    # perform search with iterables
    for tweet in twitter_api.search_tweets_iterable(tuo):
        data.append(tweet)

except TwitterSearchException as e: # catch errors
    print(e)

On peut importer le fichier **elysee.txt**, au format JSON qui contient l'ensemble des tweets de la timeline 'Elysee'

In [ ]:
elysee = json.load(open('data/elysee.txt'))

Analysons rapidement la taille ainsi que la structure des objets : 

In [ ]:
# Affichons la taille du fichier

print(len(elysee))

# Affichons les clés du premier élément

print(elysee[0].keys())

### Analyser l'évolution des tweets dans le temps

Nous allons nous intéresser dans un premier temps aux colonnes **text, retweet_count, created_at**.

Au moment de créer notre DataFrame, nous pouvons donc spécifier les colonnes à utiliser. 

Faire également attention aux Index. Nous allons travailler avec un set de données temporel, il advient de bien spécifier l'index que nous allons utiliser : Voir ***03 - Focus série temporelles avec Python***

In [ ]:
# Nous créons ici notre DataFrame

colonnes = ['text','retweet_count','created_at']

elysee_df = pd.DataFrame(elysee, 
                         columns=colonnes)

In [ ]:
# Nous transformons la colonne created_at en serie temporelle grâce à la méthode to_datetime

date = pd.to_datetime(elysee_df['created_at'])

In [ ]:
# Nous remplaçons l'index de notre DataFrame elysee_df en utilisant la méthode set_index

elysee_df = elysee_df.set_index(date)

In [ ]:
elysee_df.head()

Nous pouvons maintenant afficher le nombre de tweets par jours, en utilisant la méthode resample. 

In [ ]:
count_date = elysee_df['text'].resample('W').count()

count_date.plot().set_title('Nombre de tweets du compte @Elysee par semaine')

### A vous : 

- Afficher un graphe de l'évolution du nombre de retweets par jour. 
- Que s'est il passé en Juin 2014 et en Janvier 2015 ? 
- Afficher les tweets de la journée du 6 juin. Quel est celui qui a eu le plus de retweets?     
- Quels sont les heures les plus importantes en terme de retweets ? 

### Analyser les mentions et les hashtags les plus populaires 

Python permet également de manipuler facilement des chaines de texte grâce aux expressions régulières ***(voir Focus expression régulière dans ce dossier)*** 

Nous pouvons extraire plusieurs informations intéressantes comme : 

- Les mentions (précédés d'un '@')
- Les hastags (précédés d'un '#') 

Si nous souhaitons extraire les mentions de la colonne texte, nous pouvons écrire : 

In [ ]:
elysee_df['text'].head()

In [ ]:
pattern_mentions = "@[A-Z0-9._%+-]+"

elysee_df['mentions'] = elysee_df['text'].str.findall(pattern_mentions, re.IGNORECASE).str[0]

On peut dés lors compter le nombre de mentions en groupant par la colonne mentions :

In [ ]:
elysee_df.groupby('mentions').size().sort_values(ascending=False)[0:10]

### A vous :

   - Quelles sont les mentions les plus populaires ? 
   - Quels sont les hashtags les plus populaires ? 
   - Quels sont les hashtages les plus populaires, mentionnant @gouvernementFR ? 
   - Quelles sont les mentions les plus populaires, comportant le hashtag #DDay70 ?  